In [5]:
from functions import *
import random as rand
from scipy.optimize import fsolve



In [2]:
Perc_vax = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
niter = 5
iteraz = 0
Ring = True
alpha = 1
radius = 1
prob_vec = prob_list_powerlaw(2.5,3,400)
p = np.zeros((len(Perc_vax)+1,len(prob_vec)))
s = np.zeros((len(Perc_vax)+1,100))
nnodes = 10000
degree_vec_in = discrete_samples(prob_vec,nnodes)
graph = config_model(degree_vec_in)
p[0] = prob_vec

In [6]:
def prob_list(degree_list,n): #get a probability distribution from a given degree list
    prob_vec = []
    for i in range(n):
        prob_vec.append(degree_list.count(i)/len(degree_list))
    return prob_vec



In [9]:
def static_vax(graph,perc_vax,radius,Ring):
    while len(graph.nodes()) > nnodes*(1-perc_vax) and len(max(nx.connected_components(graph), key=len))/nnodes>0.005:
            node = rand.choice(list(graph.nodes()))
            tot_neigh = nx.single_source_shortest_path_length(graph,node,radius)
            neigh = set(key for key, value in tot_neigh.items() if value == radius) if Ring else set(key for key, value in tot_neigh.items() if value <= radius)
            for nei in neigh:
                if rand.random() < 1:
                    graph.remove_node(nei)
            #for node in tot_neigh - neigh remove edges
            if Ring:
                neigh_copia = tot_neigh.copy()
                for node in neigh_copia:
                    lista_edges_to_remove = []
                    if node not in neigh:
                        #remove all edges of node
                        lista_edges_to_remove.append([(node,neigh) for neigh in graph.neighbors(node)])
                    for i in lista_edges_to_remove:
                        graph.remove_edges_from(i)

    return graph


In [10]:

#%%

####### STATIC VAX - SIMULATION ########

while iteraz < niter:
    print(iteraz)
    j = 0
    graph_copy = graph.copy()
    for perc_vax in Perc_vax:
        j+=1
        print(j)
        
        graph_copy = static_vax(graph_copy,perc_vax,radius,Ring,alpha)

        deg_dist = [graph_copy.degree(i) for i in graph_copy.nodes()]
        prob_vec_final = prob_list(deg_dist,len(prob_vec))
        p[j] += prob_vec_final

        mean= sum([prob_vec_final[i]*i for i in range(len(prob_vec_final))])
        S2 = []
        step = np.linspace(0,1,100)
        for i in range(len(step)):
            x_intersect = fsolve(lambda x: 1-step[i]+step[i]*g_1(x,prob_vec_final,mean) - x,0.05)
            S2.append(float(list(abs((1-g_0(x_intersect,prob_vec_final)))*(1-perc_vax))[0]))
        s[j] += S2
    iteraz += 1

p = [i/iteraz for i in p]
p[0] = prob_vec

            
        

0
1
2
3
4
5
6
7
8
1
1
2
3
4
5
6
7
8
2
1
2
3
4
5
6
7
8
3
1
2
3
4
5
6
7
8
4
1
2
3
4
5
6
7
8
